In [1]:
# Notebook to retrieve and save the observed weather data

from datetime import datetime

import meteostat
import opts
import xarray as xr

from climepi._xcdat import BoundsAccessor, center_times  # noqa

# # Get options
options = opts.get_opts()
data_base_dir = options["data_base_dir"]

In [2]:
stations_all = meteostat.Stations().fetch()
station_dict = {
    "03772": "London",  # "London Heathrow Airport",
    "07149": "Paris",  # "Paris-Orly Airport",
    "17060": "Istanbul",  # "Istanbul Ataturk Airport",
    "68816": "Cape Town",  # "Cape Town International Airport",
    "72295": "Los Angeles",  # "Los Angeles International Airport",
}

for station, location in station_dict.items():
    print(
        f"For location {location}, using data from weather station: {stations_all.loc[station]['name']}"
    )

For location London, using data from weather station: London Heathrow Airport
For location Paris, using data from weather station: Paris-Orly
For location Istanbul, using data from weather station: Istanbul / Ataturk
For location Cape Town, using data from weather station: Cape Town Airport
For location Los Angeles, using data from weather station: Los Angeles Airport


In [3]:
df = meteostat.Daily(
    loc=list(station_dict),
    start=datetime(2020, 1, 1),
    end=datetime(2020, 12, 31),
).fetch()[["tavg"]]
ds = (
    xr.Dataset.from_dataframe(df)
    .rename_vars(tavg="temperature")
    .assign_coords(location=("station", list(station_dict.values())))
    .swap_dims(station="location")
    .drop_vars("station")
)

In [4]:
ds = center_times(ds.bounds.add_time_bounds(method="freq", freq="day"))
ds["time"].encoding["calendar"] = "standard"
ds["time"].encoding["units"] = "hours since 2000-01-01 00:00:00"
ds

<xarray.Dataset> Size: 24kB
Dimensions:      (location: 5, time: 366, bnds: 2)
Coordinates:
  * location     (location) <U11 220B 'London' 'Paris' ... 'Los Angeles'
  * time         (time) datetime64[ns] 3kB 2020-01-01T12:00:00 ... 2020-12-31...
Dimensions without coordinates: bnds
Data variables:
    temperature  (location, time) float64 15kB 5.3 8.7 8.6 6.4 ... 9.7 13.2 12.4
    time_bnds    (time, bnds) datetime64[ns] 6kB 2020-01-01 ... 2021-01-01

In [5]:
ds.to_netcdf(data_base_dir / "weather_2020.nc")